In [1]:
%matplotlib notebook
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
from sklearn.neighbors.kd_tree import KDTree
from ipywidgets import interactive, interact
import ipywidgets as widgets
import scipy.stats as st

In [2]:
n = 80
width = 6
height = 8

In [3]:
def plot_ann(sampling):

    if sampling == "random":
        xs = np.random.uniform(0, width, n)
        ys = np.random.uniform(0, height, n)
        pts = np.vstack((xs, ys)).T

    elif sampling == "clustered":

        centers = [(5, 3), (2, 4)]
        cluster_std = [0.3, 0.5]

        pts, c = make_blobs(n_samples=n, cluster_std=cluster_std, centers=centers, n_features=2, random_state=0)

    elif sampling == "regular":

        nx = int(np.sqrt(n * 1.9))
        X, Y = np.mgrid[0:width:complex(0, nx), 0:height:complex(0, nx)]
        pts = np.vstack([X.ravel(), Y.ravel()]).T

        idx = np.random.choice(pts.shape[0], n, replace=False)
        pts = pts[idx, :]

    mins = np.min(pts, axis=0)
    maxs = np.max(pts, axis=0)

    N = pts.shape[0]
    A = (maxs[0] - mins[0]) * (maxs[1] - mins[1])

    # Calculate expected distance
    D_e = 0.5 / np.sqrt(N / A)

    # Calculate observed distance
    tree = KDTree(pts, leaf_size=2)
    dist, _ = tree.query(pts, k=2)
    D_o = np.mean(dist, axis=0)[1]

    ANN = D_o / D_e
    
    s = 0.26136 / np.sqrt(N**2 / A)
    z_score = (D_o - D_e) / s
    
    p_value = 1.0 - st.norm.cdf(z_score) 

    fig = plt.figure(figsize=(10, 10))


    cluster_trend = "Clustering (ANN < 1)"
    if ANN > 1:
        cluster_trend = "Dispersion (ANN > 1)"

    # Z: {} P: {}
    plt.title("{}: ANN: {} --> Trend: {}".format(sampling.capitalize(),
                                                 round(ANN, 4),
                                                 cluster_trend,
                                                 round(z_score, 2),
                                                 round(p_value, 2)))


    # Data points
    plt.scatter(pts[:, 0],
                pts[:, 1],
                s=10)

    plt.xlim(mins[0] - 0.5, maxs[0] + 0.5)
    plt.ylim(mins[1] - 0.5, maxs[1] + 0.5)

    # ax = ...
    text = plt.text(0,0, "", va="bottom", ha="left")

    def onclick(event):
        tx = 'button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
        text.set_text(tx)

    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    
    plt.show()

In [4]:
interact(plot_ann,
         sampling=widgets.Dropdown(
    options=['random', 'clustered', 'regular'],
    value='clustered',
    description='Number:',
    disabled=False,
)
         );

interactive(children=(Dropdown(description='Number:', index=1, options=('random', 'clustered', 'regular'), val…